In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam
from keras.utils import np_utils
import keras
from __future__ import print_function
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
import os
import theano
from PIL import Image
from numpy import *
# SKLEARN
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

# input image dimensions
img_rows, img_cols = 200, 200

# number of channels
img_channels = 1

#  data

path1 = '/Users/antonygomez/Desktop/ML_learn/image_dataset'
#path of folder of images    
path2 = '/Users/antonygomez/Desktop/ML_learn/image_resized'
#path of folder to save images    

listing = os.listdir(path1)
num_samples=size(listing)
print(num_samples)

for file in listing:
    if file=='.DS_Store':
        continue
    else:
        im = Image.open(path1 + '//' + file)
        img = im.resize((img_rows,img_cols))
        gray = img.convert('L')
	#need to do some more processing here           
        gray.save(path2 +'//' + file, "JPEG")

imlist = os.listdir(path2)

im1 = array(Image.open('/Users/antonygomez/Desktop/ML_learn/image_resized' + '//'+ imlist[0]))
# open one image to get size
m,n = im1.shape[0:2]
# get the size of the images
imnbr = len(imlist)
# get the number of images

# create matrix to store all flattened images
immatrix = array([array(Image.open('/Users/antonygomez/Desktop/ML_learn/image_resized'+ '//' + im2)).flatten()for im2 in imlist],'f')
label=np.ones((num_samples-1,),dtype = int)
label[0:56]=0
label[56:95]=1
label[95:146]=2
label[146:197]=3
label[197:]=4
print(label.shape)
data,Label = shuffle(immatrix,label, random_state=2)
train_data = [data,Label]

Using TensorFlow backend.
/Users/antonygomez/Desktop/AnacondaInstall/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


247
(246,)


In [7]:
#batch_size to train
batch_size = 32
# number of output classes
nb_classes = 5
# number of epochs to train
nb_epoch = 30


# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 3

#%%
(X, y) = (train_data[0],train_data[1])


# STEP 1: split X and y into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)


X_train = X_train.reshape(X_train.shape[0],img_rows, img_cols,1)
X_test = X_test.reshape(X_test.shape[0],img_rows, img_cols,1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = Sequential()

model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                        border_mode='valid',
                        input_shape=(img_rows, img_cols,1)))
convout1 = Activation('relu')
model.add(convout1)
model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
convout2 = Activation('relu')
model.add(convout2)
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',metrics=['accuracy'])
         
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,verbose=1, validation_split=0.2)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save('keras_cnn_model_our_dataset.h5')


X_train shape: (196, 200, 200, 1)
196 train samples
50 test samples


/Users/antonygomez/Desktop/AnacondaInstall/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:52: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(200, 200,..., padding="valid")`
/Users/antonygomez/Desktop/AnacondaInstall/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
/Users/antonygomez/Desktop/AnacondaInstall/anaconda3/lib/python3.6/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 156 samples, validate on 40 samples
Epoch 1/30
156/156 [==============================] - 29s - loss: 4.7228 - acc: 0.1731 - val_loss: 1.6008 - val_acc: 0.2500
Epoch 2/30
156/156 [==============================] - 26s - loss: 1.6299 - acc: 0.2564 - val_loss: 1.5666 - val_acc: 0.2750
Epoch 3/30
156/156 [==============================] - 26s - loss: 1.5915 - acc: 0.2372 - val_loss: 1.5780 - val_acc: 0.2750
Epoch 4/30
156/156 [==============================] - 28s - loss: 1.5886 - acc: 0.2564 - val_loss: 1.5492 - val_acc: 0.2750
Epoch 5/30
156/156 [==============================] - 28s - loss: 1.4978 - acc: 0.2821 - val_loss: 1.5341 - val_acc: 0.3250
Epoch 6/30
156/156 [==============================] - 28s - loss: 1.5995 - acc: 0.2692 - val_loss: 1.5965 - val_acc: 0.2500
Epoch 7/30
156/156 [==============================] - 27s - loss: 1.5747 - acc: 0.2692 - val_loss: 1.5773 - val_acc: 0.1750
Epoch 8/30
156/156 [==============================] - 25s - loss: 1.4704 - acc: 0.3333 